In [1]:
import pandas as pd
print(f'Pandas version: {pd.__version__}')
import sklearn
print(f'sklearn version: {sklearn.__version__}')

Pandas version: 2.2.2
sklearn version: 1.4.2


## Q1. Downloading the data
Download the data for January and February 2023.
Read the data for January. How many columns are there?


In [2]:
df_01 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_02 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
print(f'There are {len(df_01.columns)} columns')

There are 19 columns


## Q2. Computing duration 
Now let's compute the __duration__ variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [3]:
df_01.columns
df_01['duration'] = df_01.tpep_dropoff_datetime - df_01.tpep_pickup_datetime
df_01['duration_sec'] = df_01['duration'].apply(lambda td: td.total_seconds())
df_01['duration_min'] = df_01['duration'].apply(lambda td: td.total_seconds() * 1. / 60)
print(f"Std is: {df_01['duration_min'].std():.2f}")

Std is: 42.59


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
What fraction of the records left after you dropped the outliers?

In [4]:
filtered_df = df_01[(df_01['duration_min'] >= 1) & (df_01['duration_min'] <= 60)]

# Calculate the fraction of records left after filtering
fraction_left = len(filtered_df) / len(df_01)
print(f"Filtered DataFrame is {fraction_left * 100:.2f}% of whole df")

Filtered DataFrame is 98.12% of whole df
